# Assignment 3 

# Table of Content
## Overview
1. Where is 307?

## Data Exploration
1. People's Behavior in terms of Dwell Time 
2. Which areas of 307 do people pass through
3. Where do people tend to linger?
4. How does dwell time change over time?

## In-depth Analysis
1. How do different zones affect people's behavior?
2. How do events affect people's behavior?
3. What is the best maintenance strategy?
4. What are other factor affect people's bahavior?

# About 307

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

In [2]:
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
# pd.options.display.max_rows = 30
# pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
# store login data in login.py
%run login.py

In [5]:
# login query as multiline formatted string
# this assumes that login and pwd are defined 
# above

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

In [6]:
import requests
url = 'https://api.numina.co/graphql'

mylogin = requests.post(url, json={'query': loginquery})
# mylogin

In [7]:
token = mylogin.json()['data']['logIn']['jwt']['token']

In [8]:
expdate = mylogin.json()
# expdate

# Explore the Data!

Now that you've been provided with the context, before we present our analysis, it's time for YOU to explore the data! As mentioned, the following are the full areas covered by the three cameras:

Streetscape | Under Raincoat | Outside
------------- | -------------  | -------------
![alt](streetscape_sandbox.png) | ![alt](underraincoat_sandbox.png) | ![alt](outside_sandbox.png)

As you see in the above images, each area essentially consists of two parts: objects such as tables and chairs, and empty spaces presumably for walking. Based on this reasoning, we have defined the following smaller behaviour zones so as to perform more in-depth research:

### Streetscape ###

Chair Zone | Corridor Zone | Free Zone
------------- | -------------  | -------------
![alt](BehaviorZoneImage/Streetscape-ChairZone.png) | ![alt](BehaviorZoneImage/Streetscape-PathZone.png) | ![alt](BehaviorZoneImage/Streetscape-ActivityZone.png)

### Under Raincoat ###

Chair Zone | Traffic Zone | Free Zone
------------- | -------------  | -------------
![alt](BehaviorZoneImage/UnderRaincoat-ChairZone.png) | ![alt](BehaviorZoneImage/UnderRaincoat-TrafficZone.png) | ![alt](BehaviorZoneImage/UnderRaincoat-ActivityZone.png)

### Outside ###

Chair Zone | Path Zone | -
------------- | -------------  | -------------
![alt](BehaviorZoneImage/Outside-ChairZone.png) | ![alt](BehaviorZoneImage/Outside-PathZone.png) | ![alt](blank.png)

Note that we have to be aware of the fact that the chairs can be moved and that the above images may not necessarily reflect the layout of the room during the whole period of data collection. Specifically, the three sets of chairs in the Under Raincoat area can be easily moved; thus in the initial exploration, we will not be investigating the Chair Zone of Under Raincoat. 

Nonetheless, notice that they are included in the Free Zone. We believe that it is safe to assume that the chairs would not be moved outside the Free Zone to the Traffic Zone.

Similarly, in the Streetscape area, under the assumption that it is intended to place the chairs together, it is unlikely that the group of chairs would be moved around freely and frequently due to the other obstacles in the room. As for the Outside area, it is also unlikely that the chairs would be placed in the middle of the road to block the path. Thus, we will be analyzing these two Chair Zones (while keeping the limitation in mind).

In [9]:
device_dict = {'SWLSANDBOX1':'Streetscape', 'SWLSANDBOX2':'Under Raincoat', 'SWLSANDBOX3':'Outside'}
device_ids = list(device_dict.keys())
device_names = list(device_dict.values())

# streetscape, under raincoat, outside
device_clrs = ['royalblue', 'firebrick', 'forestgreen']

In [10]:
def get_zones(device_id):
    
    query_zones = """
    query {{
      behaviorZones (
        serialnos: "{0}"
        ) {{
        count
        edges {{
          node {{
            rawId
            text
          }}
        }}
      }}
    }}
    """.format(device_id)
    
    zones = requests.post(url, json={'query': query_zones}, headers = {'Authorization':token})
    
    df = pd.DataFrame([x['node'] for x in zones.json()['data']['behaviorZones']['edges']])
    df['device_id'] = device_id
    
    return df

In [11]:
zones_df = pd.concat([get_zones(device_ids[i]) for i in range(3)])
zones_df = zones_df[(zones_df.text.notnull()) & 
                    (zones_df.text.str.startswith('x-')) & 
                    (zones_df.text.str.endswith('zone'))]

In [12]:
zones_df['text'] = zones_df['text'].str.replace('x-', '')
zones_df['type'] = ['path', 'rest', 'both', 'path', 'both', 'rest', 'path']

# zone ID from int to str
zones_df.rawId = zones_df.rawId.astype(str)
zone_name_dict = dict(zip(zones_df.rawId, zones_df.text))
zone_type_dict = dict(zip(zones_df.rawId, zones_df.type))

In [13]:
def get_dwell(func, ID, interval):
    '''
    func is either feedDwellTimeDistribution or zoneDwellTimeDistribution
    '''
    if func == 'feedDwellTimeDistribution':
        arg = 'serialnos: "{0}"'.format(ID)
    else:
        arg = 'zoneIds: {0}'.format(ID)
        
    query = """
    query {{
        {0}(
        {1},
        startTime: "2019-02-20T00:00:00",
        endTime: "2020-01-12T00:00:00",
        timezone: "America/New_York",
        objClasses: ["pedestrian"],
        interval: "{2}"
        ){{
        edges {{
          node {{
            time
            objClass
            pct100
            pct75
            pct50
            pct25
            mean
            count
          }}
        }}
      }}
    }}
    """.format(func, arg, interval)

    dwell = requests.post(url, json={'query': query}, 
                           headers = {'Authorization':token})
    
    df = pd.DataFrame([x['node'] for x in dwell.json()['data'][func]['edges']])
    if func == 'feedDwellTimeDistribution':
        df['device_id'] = ID
    else:
        df['zone_id'] = ID
    
    return df

In [14]:
def preprocess(df):
    # replace NaN with 0
    df = df.fillna(0)
    # convert time
    df['time'] = df['time'].str[:-6].apply(lambda x : pd.Timestamp(x))
    df['month'] = df['time'].dt.month
    df['dayofweek'] = df['time'].dt.dayofweek
    df['hour'] = df['time'].dt.hour
    df['date'] = df['time'].dt.date
    
    # add either zone or device name
    if 'zone_id' in df.columns:
        df.zone_id = df.zone_id.astype(str)
        df['zone'] = [zone_name_dict[z] for z in df.zone_id]
        df['zone_type'] = [zone_type_dict[z] for z in df.zone_id]
    else:
        df['device'] = [device_dict[d] for d in df.device_id]
    
    # add a total column = mean * count
    df['total_dwell'] = df['mean'] * df['count']
    df = df.rename(columns={'mean':'mean_dwell', 'pct50':'median_dwell', 'pct100':'max_dwell'})
    df = df.drop(['pct75', 'pct25'], axis=1)
    
    return df

In [15]:
# hourly dwell time 
# device
feed_dwell_1h_df = pd.concat([get_dwell('feedDwellTimeDistribution', device_ids[i], '1h') 
                              for i in range(3)])
# zone
zone_dwell_1h_df = pd.concat([get_dwell('zoneDwellTimeDistribution', z, '1h')
                             for z in zones_df['rawId'].values])

feed_dwell_1h_df = preprocess(feed_dwell_1h_df)
zone_dwell_1h_df = preprocess(zone_dwell_1h_df)

In [16]:
# daily dwell time 
# device
feed_dwell_1d_df = pd.concat([get_dwell('feedDwellTimeDistribution', device_ids[i], '1d') 
                              for i in range(3)])
# zone
zone_dwell_1d_df = pd.concat([get_dwell('zoneDwellTimeDistribution', z, '1d')
                             for z in zones_df['rawId'].values])

feed_dwell_1d_df = preprocess(feed_dwell_1d_df)
zone_dwell_1d_df = preprocess(zone_dwell_1d_df)

In [96]:
# colour of zones - 3 blues, 2 reds, 2 greens
zone_clrs = ['royalblue', 'deepskyblue', 'dodgerblue',
             'lightcoral', 'orangered', 
             'mediumaquamarine', 'mediumseagreen']

In [18]:
def get_df(groupby, interval):
    if groupby == 'device' and interval == '1d':
        return feed_dwell_1d_df.copy(), device_names
    elif groupby == 'zone' and interval == '1d':
        return zone_dwell_1d_df.copy(), list(zones_df.text)
    elif groupby == 'device' and interval == '1h':
        return feed_dwell_1h_df.copy(), device_names
    elif groupby == 'zone' and interval == '1h':
        return zone_dwell_1h_df.copy(), list(zones_df.text)

Recall that the timeframe of our data is approximately one year. Therefore, in the initial exploration, let's focus on the daily dwell time and daily count of pedestrains in the 307 region. 

As a starting point, explore the data using the following interactive line plot and think about these questions:
1. Is there any trend in pedesdrian count / dwell time in any of the areas / zones?
2. Where would you expect to see a bigger crowd? Is any of the areas / zones more popular than others?

Tip: You can click the legend on the right to include/exclude a line on the plot.

In [19]:
metric_list = ['count', 'mean_dwell', 'max_dwell', 'median_dwell', 'total_dwell']

In [20]:
def plot_byhour(groupby, metric, quantiles):
    # byhour
    df, byvals = get_df(groupby, '1h')
    
    # filter based on quantiles
    df = df[(df[metric] <= df[metric].quantile(quantiles[1])) &
            (df[metric] >= df[metric].quantile(quantiles[0]))].sort_values('hour')
    
    df['-'] = '-'
    # plot differently based on whethere device or zone
    title = 'change in '+metric+' over different times of the day, with point size being count'
    # device
    if groupby=='device':
        fig = px.scatter(df, x='-', y=metric, color='device', facet_col='device', size='count',
                     animation_frame='hour', opacity=0.5, size_max=30,
                     title=title)
    # zone
    else:
        df['device'] = df['zone'].apply(lambda x : x.split('-')[0])
        fig = px.scatter(df, y=metric, x='device', color='device', facet_col='zone_type', size='count',
                     animation_frame='hour', opacity=0.5, size_max=30,
                     title=title)
        
    # labels
    #fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
    fig.layout.update(showlegend=True)
    fig.update_xaxes(showticklabels=False)
    fig.update_xaxes(title='')
    fig.show()

In [21]:
_ = interact(plot_byhour, groupby=widgets.RadioButtons(options=['device', 'zone'], value='zone'),
             metric=widgets.Dropdown(options=metric_list[1:], value=metric_list[1]),
             quantiles=widgets.FloatRangeSlider(value=[0, 0.98], min=0, max=1, step=0.01, continuous_update=False))

interactive(children=(RadioButtons(description='groupby', index=1, options=('device', 'zone'), value='zone'), …

In [93]:
px.scatter_matrix(feed_dwell_1d_df, ['count', 'mean_dwell', 'dayofweek', 'month'], color='device', opacity=0.5)

In [116]:
def plot_avg(groupby, metric, timegroup):
    '''
    plots metric of each groupby ('device' or 'zone') 
    by timegroup ('hour', 'date', 'dayofweek', 'month')
    '''
    # get either 1d or 1h depending on time group
    if timegroup == 'hour':
        df, byvals = get_df(groupby, '1h')
    else:
        df, byvals = get_df(groupby, '1d')
    
    # colours
    if groupby == 'device':
        # default colours
        clr = None
    else:
        clr = zone_clrs
        
    # order of dayofweek
    weekdays = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    df['dayofweek'] = [weekdays[w] for w in df['dayofweek']]
    
    # filter out 0 count day/hour
    m = df.groupby('time').sum()
    df = df[df['time'].isin(list(m[m['count']!=0].index))]
    # group
    df = df.groupby([timegroup, groupby]).median().reset_index()
    
    # plot
    fig = px.bar(df, x=timegroup, y=metric, color=groupby,
                 category_orders={groupby:byvals, 'dayofweek':list(weekdays.values())}, # order of categories
                 color_discrete_sequence=clr, # colour of devices/zones
                 title='median of pedestrian ' + metric + ' grouped by ' + timegroup + ' and ' + groupby,
                 barmode='group')
    fig.show()

In [117]:
_ = interact(plot_avg, 
             groupby=widgets.RadioButtons(options=['device', 'zone'], value='device'),
             metric=widgets.Dropdown(options=metric_list, value='mean_dwell'),
             timegroup=widgets.Dropdown(options=['hour', 'dayofweek', 'month'], value='hour'))

interactive(children=(RadioButtons(description='groupby', options=('device', 'zone'), value='device'), Dropdow…

In [62]:
def compute_prop_helper(df, col, newcol, lower_bound):
    '''
    helper function for compute_prop;
    newcol is proportion wrt col
    '''
    # group by time (date + hour)
    m = df.groupby('time').sum()
    
    # compute proportion wrt the total at each time
    # add 0.00001 to avoid division by zero
    df[newcol] = df.apply(lambda x : x[col] / (0.00001 + m.loc[x['time'], col]), axis=1)
    
    # only keep the times where the total count/dwell time is at least lower_bound
    sub = df[df['time'].isin(list(m[m['count']>=lower_bound].index))]
    
    return sub

In [63]:
def compute_prop(df, lower_bound):
    '''
    computes proportion of count and total_dwell wrt the total at the time;
    only keep days/hours in which the total value is at least lower_bound;
    adds two new columns - count_prop and total_dwell_prop
    '''
    df = df.copy()
    df = compute_prop_helper(df, 'count', 'count_prop', lower_bound)
    df = compute_prop_helper(df, 'total_dwell', 'total_dwell_prop', lower_bound)
    return df

In [120]:
def plot_prop(groupby, metric, timegroup, lower_bound):
    '''
    plots proportion of metric ('count' or 'total_dwell')
    of each groupby ('device' or 'zone') 
    by timegroup ('hour', 'date', 'dayofweek', 'month');
    lower_bound is on the total value at a time
    '''
    # get either 1d or 1h depending on time group
    if timegroup == 'hour':
        df, byvals = get_df(groupby, '1h')
    else:
        df, byvals = get_df(groupby, '1d')
    
    # colours
    if groupby == 'device':
        # default colours
        clr = None
    else:
        clr = zone_clrs
        
    # add proportion columns
    df = compute_prop(df, lower_bound)
    # group
    df = df.groupby([timegroup, groupby]).mean().reset_index()
    
    # order of dayofweek
    weekdays = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    df['dayofweek'] = [weekdays[w] for w in df['dayofweek']]
    
    # plot
    fig = px.bar(df, x=timegroup, y=metric+'_prop', color=groupby,
                 category_orders={groupby:byvals, 'dayofweek':list(weekdays.values())}, # order of categories
                 color_discrete_sequence=clr, # colour of devices/zones
                 title='averaged proportion of ' + metric + ' accounted for by each ' + groupby,
                 range_y=(0, 1))
    fig.show()

In [121]:
_ = interact(plot_prop, 
             groupby=widgets.RadioButtons(options=['device', 'zone'], value='device'),
             metric=widgets.RadioButtons(options=['count', 'total_dwell'], value='count'),
             timegroup=widgets.Dropdown(options=['hour', 'date', 'dayofweek', 'month'], value='dayofweek'),
             lower_bound=widgets.IntSlider(value=50, min=0, max=2000, step=50, continuous_update=False))

interactive(children=(RadioButtons(description='groupby', options=('device', 'zone'), value='device'), RadioBu…

In [72]:
def plot_timeline(groupby, metric):
    '''
    groupby is either 'device' or 'zone';
    metric is a value in metric_list
    '''
    df, byvals = get_df(groupby, '1d')
    
    fig = go.Figure()
    
    # line plot for each name
    for i in range(len(byvals)):
        sub_df = df[df[groupby] == byvals[i]]
        fig.add_trace(go.Scatter(x=sub_df.time, y=sub_df[metric], name=byvals[i]))
    
    # layout - axes labels
    fig.update_layout(
        xaxis_title="time",
        yaxis_title=metric,
        xaxis_rangeslider_visible=True
    )
    # title
    if metric != 'count':
        fig.update_layout(title=f"pedestrian dwell time ({metric}) grouped by '{groupby}'")
    else:
        fig.update_layout(title=f"pedestrian count grouped by '{groupby}'")
    
    fig.show()
    

In [79]:
def plot_timeline(groupby, metric):
    '''
    groupby is either 'device' or 'zone';
    metric is a value in metric_list
    '''
    df, _ = get_df(groupby, '1d')
    
    fig = px.line(df, x='time', y=metric, color=groupby, title='pedestrian '+metric+' grouped by '+groupby)
    
    # layout - axes labels
    fig.update_layout(
        xaxis_rangeslider_visible=True
    )
    
    fig.show()
    

In [80]:
_ = interact(plot_timeline, 
             groupby=widgets.RadioButtons(options=['device', 'zone'], value='device'),
             metric=widgets.Dropdown(options=metric_list, value='median_dwell')
            )

interactive(children=(RadioButtons(description='groupby', options=('device', 'zone'), value='device'), Dropdow…

Not too surprisingly, we observe a few peak days. The following interactive dataframe summarizes the exact locations and dates:

In [74]:
def sort_dwell_1d(groupby, sortby, ascending, top):
    df, _ = get_df(groupby, '1d')
    
    cols = [groupby, 'time', sortby]
    if sortby == 'count':
        cols.append('mean_dwell')
    elif sortby == 'mean_dwell':
        cols.append('count')
    else:
        cols.append('count')
        cols.append('mean_dwell')
        
    display(df.sort_values(sortby, ascending=ascending).reset_index(drop=True)
              .loc[:int(top)-1, cols])

_ = interact(sort_dwell_1d, 
             groupby=widgets.RadioButtons(options=['device', 'zone'], value='device'),
             sortby=widgets.Dropdown(options=metric_list, value='mean_dwell'),
             top=widgets.IntSlider(value=5, min=1, max=30, step=1, readout_format='d'),
             ascending=widgets.Checkbox(value=False, description='ascending'))

interactive(children=(RadioButtons(description='groupby', options=('device', 'zone'), value='device'), Dropdow…

In [89]:
def plot_boxplot(groupby, metric):
    fig = go.Figure()
    
    df, byvals = get_df(groupby, '1d')
    
    for i in range(len(byvals)):
        # Use x instead of y argument for horizontal plot
        fig.add_trace(go.Box(x=df.loc[df[groupby]==byvals[i], metric], name=byvals[i],
                             boxpoints='outliers'))

    # layout - axes labels
    fig.update_layout(
        xaxis_title=metric,
        xaxis_rangeslider_visible=True
    )
    # title
    if metric != 'count':
        fig.update_layout(title=f"distribution of pedestrian dwell time ({metric}) grouped by '{groupby}'")
    else:
        fig.update_layout(title=f"distribution of pedestrian count grouped by '{groupby}'")
    
    fig.show()
    

In [90]:
_ = interact(plot_boxplot, 
             groupby=widgets.RadioButtons(options=['device', 'zone'], value='device'),
             metric=widgets.Dropdown(options=metric_list, value='mean_dwell')
            )

interactive(children=(RadioButtons(description='groupby', options=('device', 'zone'), value='device'), Dropdow…

### Obtain heatmap for pedestrians

In [36]:
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
import calendar
START_DATE = datetime(2019, 2, 20, 0, 0, 0)
END_DATE = datetime(2019, 3, 20, 0, 0, 0)
time_delta = relativedelta(days = +1)

In [37]:
import pandas as pd
heatmap_df = pd.DataFrame(columns = ['startTime', 'endTime', 'heatMap'])

In [38]:
def heatmap_query_gen(startTime: str, endTime: str):
    heatmap_query = """
query {{
  feedHeatmaps(
    serialno: "SWLSANDBOX1",
    startTime:"{0}",
    endTime:"{1}",
    objClasses:["pedestrian"],
    timezone:"America/New_York") {{
    edges {{
      node {{
        time
        objClass
        heatmap
      }}
    }}
  }}
}}
""".format(startTime, endTime)
    return heatmap_query

In [39]:
current_date = START_DATE
while current_date < END_DATE:
    start_time_str = current_date.strftime('%Y-%m-%dT%H:%M:%S')
    end_time = current_date + time_delta
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S')
    heatmap_data = requests.post(url, json={'query': heatmap_query_gen(start_time_str, end_time_str)}, 
                         headers = {'Authorization':token})
    heatmap_json = heatmap_data.json()
    if heatmap_json['data']:
        if 'feedHeatmaps' in heatmap_json['data']:
            heatmap = heatmap_json['data']['feedHeatmaps']['edges'][0]['node']['heatmap']
            temp_df = pd.DataFrame({"startTime":current_date, "endTime":end_time, 'heatMap':heatmap})
            heatmap_df = heatmap_df.append(temp_df, ignore_index = True)
    current_date = current_date + time_delta

In [40]:
ed_heatmap_df = heatmap_df.groupby(['startTime', 'endTime'])['heatMap'].apply(list).reset_index(name='heatMapMatrix')

In [41]:
from IPython.display import display
def plot_heatmap(start_time):
    map_img = mpimg.imread('streetscape_sandbox.png')
    matrix = list(ed_heatmap_df[ed_heatmap_df['startTime'] == start_time]['heatMapMatrix'])[0]
    x = [i[0] for i in matrix] 
    y = [i[1] for i in matrix]
    z = [i[2] for i in matrix]
    fig, ax = plt.subplots(figsize=(15,10))
    ax.scatter(x, y, c=z, s=10, cmap=plt.cm.Wistia) # Other color maps: plt.cm.cmap_d.keys())
    ax.imshow(map_img, aspect='auto')
    plt.axis('off')
    plt.title("Heatmap for date {0}".format(start_time, fontsize=20))
    plt.show()
interact(plot_heatmap, start_time=widgets.DatePicker(value = pd.to_datetime('2019-02-26'), description='Pick a Date'))

interactive(children=(DatePicker(value=Timestamp('2019-02-26 00:00:00'), description='Pick a Date'), Output())…

<function __main__.plot_heatmap(start_time)>

## Event vs Non Event Days

### Subsection: Pedestrian Count

In this section, we will be exploring how poeple's behaviour differ when there is an event and when there is no event occuring. We have obtained the Sidewalk Labs' event schedule from the [website](https://www.sidewalktoronto.ca/participate/). I have recorded all the events between Febuary 20th, 2019 and January 11th, 2020.

In [42]:
event_dates = pd.read_csv('EventDates.csv')

I also have obtained the pedestrian count data. We will first explore how the pedestrian count changes in different days. 

In [43]:
outside_count_df = pd.read_csv('OverviewForOutsideCount.csv')
streetscape_count_df = pd.read_csv('OverviewForStreetScapeCount.csv')
under_rain_coat_count_df = pd.read_csv('OverviewForUnderRainCoatCount.csv')

FileNotFoundError: [Errno 2] File b'OverviewForOutsideCount.csv' does not exist: b'OverviewForOutsideCount.csv'

In [ ]:
outside_count_df.time = outside_count_df.time.str[:-6]
streetscape_count_df.time = streetscape_count_df.time.str[:-6]
under_rain_coat_count_df.time = under_rain_coat_count_df.time.str[:-6]

In [ ]:
from datetime import datetime as dt
outside_count_df.time = outside_count_df.apply(lambda x: dt.strptime(x.time, '%Y-%m-%dT%H:%M:%S'), axis = 1)
streetscape_count_df.time = streetscape_count_df.apply(lambda x: dt.strptime(x.time, '%Y-%m-%dT%H:%M:%S'), axis = 1)
under_rain_coat_count_df.time = under_rain_coat_count_df.apply(lambda x: 
                                                               dt.strptime(x.time, '%Y-%m-%dT%H:%M:%S'), axis = 1)

In [ ]:
outside_count_by_day = outside_count_df.resample('d', on='time')['pedestrians'].agg(np.sum)
streetscape_count_by_day = streetscape_count_df.resample('d', on='time')['pedestrians'].agg(np.sum)
under_rain_coat_count_df_by_day = under_rain_coat_count_df.resample('d', on='time')['pedestrians'].agg(np.sum)

In [ ]:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x=outside_count_by_day.index, y=outside_count_by_day.values, 
                         name="Outside",
                         line_color='royalblue'))

fig = fig.add_trace(go.Scatter(x=streetscape_count_by_day.index, y=streetscape_count_by_day.values, 
                         name="Street Scape",
                         line_color='dimgray'))

fig = fig.add_trace(go.Scatter(x=under_rain_coat_count_df_by_day.index, y=under_rain_coat_count_df_by_day.values, 
                         name="Under Rain Coat",
                         line_color='firebrick'))

fig = fig.update_layout(title_text='Pedestrians Count By Day',
                  xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
sum_ped_count_by_day = outside_count_by_day + streetscape_count_by_day + under_rain_coat_count_df_by_day

In [ ]:
sum_ped_count_by_day

In [ ]:
# fig = go.Figure(boxpoints='all')
# fig.add_trace(go.Box(x=sum_ped_count_by_day.values))

From the time series line plot above, we notced that there are several days that have significantly higher pedestrian count than other days. We will examine this further in the hour granular level.

In [ ]:
outside_count_by_hour = outside_count_df.resample('H', on='time')['pedestrians'].agg(np.sum)
streetscape_count_by_hour = streetscape_count_df.resample('H', on='time')['pedestrians'].agg(np.sum)
rain_coat_count_by_hour = under_rain_coat_count_df.resample('H', on='time')['pedestrians'].agg(np.sum)

In [ ]:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x=outside_count_by_hour.index, y=outside_count_by_hour.values, 
                         name="Outside",
                         line_color='royalblue'))

fig = fig.add_trace(go.Scatter(x=streetscape_count_by_hour.index, y=streetscape_count_by_hour.values, 
                         name="Street Scape",
                         line_color='dimgray'))

fig = fig.add_trace(go.Scatter(x=rain_coat_count_by_hour.index, y=rain_coat_count_by_hour.values, 
                         name="Under Rain Coat",
                         line_color='firebrick'))

fig = fig.update_layout(title_text='Pedestrians Count By Hour',
                  xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
def plot_pedestrian_count_event(event):
    '''
    Display time series pedestrian count of the event specified
    '''
    event_info = event_dates[event_dates.Event == event]
    start = dt.strptime(event_info['Starting Time'].values[0], '%Y-%m-%dT%H:%M:%S')
    end = dt.strptime(event_info['Ending Time'].values[0], '%Y-%m-%dT%H:%M:%S')
    
    outside = outside_count_by_hour[(outside_count_by_hour.index >= start) & (outside_count_by_hour.index <= end)]
    streetscape = streetscape_count_by_hour[(streetscape_count_by_hour.index >= start) & \
                                            (streetscape_count_by_hour.index <= end)]
    rain_coat = rain_coat_count_by_hour[(rain_coat_count_by_hour.index >= start) & \
                                        (rain_coat_count_by_hour.index <= end)]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=outside.index, y=outside.values, 
                         name="Outside",
                         line_color='royalblue'))

    fig.add_trace(go.Scatter(x=streetscape.index, y=streetscape.values, 
                         name="Street Scape",
                         line_color='dimgray'))

    fig.add_trace(go.Scatter(x=rain_coat.index, y=rain_coat.values, 
                         name="Under Rain Coat",
                         line_color='firebrick'))
    
    fig.add_trace(go.Scatter(x=rain_coat.index, y=outside.values+streetscape.values+rain_coat.values, 
                         name="Sum",
                         line_color='gold'))

    fig.update_layout(title_text=f'Pedestrians Count By Event: {event}',
                  xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
_ = interact(plot_pedestrian_count_event, 
             event=widgets.Dropdown(options=event_dates.Event.tolist(), value='Sidewalk Summer Open House'),
            )

### Subsection: Dwell Time

In [ ]:
# daily dwell time - device
feed_dwell_1h_df = pd.concat([get_dwell('feedDwellTimeDistribution', device_ids[i], '1h') 
                              for i in range(3)])

In [ ]:
feed_dwell_1h_df = feed_dwell_1h_df.dropna()

In [ ]:
feed_dwell_1h_df.time = feed_dwell_1h_df.time.str[:-6]

In [ ]:
feed_dwell_1h_df['device_name'] = [device_dict[d] for d in feed_dwell_1h_df.device]

In [ ]:
feed_dwell_1h_df.time = feed_dwell_1h_df.apply(lambda x: dt.strptime(x.time, '%Y-%m-%dT%H:%M:%S'), axis = 1)

In [ ]:
def plot_dwell_time_event(event, metric):
    '''
    Display time series of the matrix of the event specified
    '''
    event_info = event_dates[event_dates.Event == event]
    start = dt.strptime(event_info['Starting Time'].values[0], '%Y-%m-%dT%H:%M:%S')
    end = dt.strptime(event_info['Ending Time'].values[0], '%Y-%m-%dT%H:%M:%S')

    dwell_time_df = feed_dwell_1h_df[(feed_dwell_1h_df.time >= start) & (feed_dwell_1h_df.time <= end)]
    
    outside = dwell_time_df[dwell_time_df.device_name == 'Outside']
    streetscape = dwell_time_df[dwell_time_df.device_name == 'Streetscape']
    rain_coat = dwell_time_df[dwell_time_df.device_name == 'Under Raincoat']
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=outside.time, y=outside[metric], 
                         name="Outside",
                         line_color='royalblue'))

    fig.add_trace(go.Scatter(x=streetscape.time, y=streetscape[metric], 
                         name="Street Scape",
                         line_color='dimgray'))

    fig.add_trace(go.Scatter(x=rain_coat.time, y=rain_coat[metric], 
                         name="Under Rain Coat",
                         line_color='firebrick'))
    
    fig.update_layout(title_text=f"Distribution of Pedestrian dwell time ({metric}) By Event: {event}",
                  xaxis_rangeslider_visible=True)
    
    fig.update_layout(
        xaxis_title="time",
        yaxis_title=metric,
    )
    
    fig.show()
    
    

In [ ]:
_ = interact(plot_dwell_time_event, 
             event=widgets.Dropdown(options=event_dates.Event.tolist(), value='Sidewalk Summer Open House'),
             metric=widgets.Dropdown(options=['mean', 'pct100', 'pct75', 'pct50', 'pct25', 'total'], 
                                     value='mean')
            )

## Maintenance Strategy

In [44]:
# need hourly data so writing the query again; can combine with the previous one later
def get_dwell_by_hour(func, ID):
    '''
    func is either feedDwellTimeDistribution or zoneDwellTimeDistribution
    '''
    if func == 'feedDwellTimeDistribution':
        arg = 'serialnos: "{0}"'.format(ID)
    else:
        arg = 'zoneIds: {0}'.format(ID)
        
    query = """
    query {{
        {0}(
        {1},
        startTime: "2019-02-20T00:00:00",
        endTime: "2020-01-12T00:00:00",
        timezone: "America/New_York",
        objClasses: ["pedestrian"],
        interval: "1h"
        ){{
        edges {{
          node {{
            time
            objClass
            pct100
            pct75
            pct50
            pct25
            mean
            count
          }}
        }}
      }}
    }}
    """.format(func, arg)

    dwell = requests.post(url, json={'query': query}, 
                           headers = {'Authorization':token})
    
    df = pd.DataFrame([x['node'] for x in dwell.json()['data'][func]['edges']])
    if func == 'feedDwellTimeDistribution':
        df['device'] = ID
    else:
        df['zone'] = ID
    
    return df

In [45]:
feed_dwell_df = pd.concat([get_dwell_by_hour('feedDwellTimeDistribution', device_ids[i]) 
                           for i in range(3)])

In [46]:
# replace NaN with 0
feed_dwell_df = feed_dwell_df.fillna(0)

# convert time to timestamp object
feed_dwell_df['time'] = feed_dwell_df['time'].str[:-6].apply(lambda x : pd.Timestamp(x))

# add name column in addition to ID
feed_dwell_df['device_name'] = [device_dict[d] for d in feed_dwell_df.device]

In [47]:
import datetime as dt
from pandas.api.types import CategoricalDtype
days = [(dt.datetime(2019, 3, 4) + dt.timedelta(days=x)).strftime('%a') for x in range(0, 7)]
day_type = CategoricalDtype(categories=days, ordered=True)

feed_dwell_df['day of week'] = feed_dwell_df['time'].apply(lambda x: x.strftime('%a')).astype(day_type)
feed_dwell_df['date'] = feed_dwell_df['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
feed_dwell_df['hour'] = feed_dwell_df['time'].apply(lambda x: x.strftime('%H'))
feed_dwell_df['hour'] = pd.to_numeric(feed_dwell_df['hour'])

In [55]:
daily_count = feed_dwell_df.groupby(['date', 'device_name'])['count'].max()
daily_count = pd.DataFrame(daily_count).reset_index()
daily_count['date'] = pd.to_datetime(daily_count['date'])

In [49]:
def plot_count(selected, start_date, end_date, threshold):
    '''
    device_or_zone is either 'device' or 'zone';
    selected is a list of device rawIds or zone rawIds;
    metric is a value in ['mean', 'pct100', 'pct75', 'pct50', 'pct25']
    '''
    #df = nonzero_df
    df = daily_count
        
    plot_df = df.loc[(df.date >= pd.Timestamp(start_date)) & 
                     (df.date <= pd.Timestamp(end_date))].copy()
    
    fig = go.Figure()
    
    for device in selected:
        sub_df = plot_df[plot_df['device_name'] == device]
        sub_df_under = sub_df[sub_df['count'] <= threshold]
        fig.add_trace(go.Scatter(x=sub_df_under.date, y=sub_df_under['count'], mode='lines', name=device))
        # TODO: fix string representation
        print("There are", len(sub_df_under), "days for ", device, 
              "with a daily pedestrian count under", threshold)
        print("There are", len(sub_df)-len(sub_df_under), "days for ", device, 
              "with a daily pedestrian count above", threshold)
    
    fig.update_layout(
        title="Pedestrian count under threshold grouped by device",
        xaxis_title="time",
        yaxis_title="count")
    
    fig.show()

In [50]:
# SWLSANDBOX1 = Streetscape
# SWLSANDBOX2 = Under Raincoat
# SWLSANDBOX3 = Outside
_ = interact(plot_count, 
             selected=widgets.SelectMultiple(options=device_names, value=device_names, disabled=False),
             start_date=widgets.DatePicker(value=pd.to_datetime('2019-02-20')),
             end_date=widgets.DatePicker(value=pd.to_datetime('2020-01-12')),
             threshold=widgets.IntSlider(value=500, min=300, max=1000, step=100, readout_format='d')
            )

interactive(children=(SelectMultiple(description='selected', index=(0, 1, 2), options=('Streetscape', 'Under R…

In [51]:
# TODO: combine the two box plots to a single interactive
def plot_boxplot_count_by_day(threshold):
    fig = go.Figure()
    
    #df, byvals, clrs = get_df(groupby)
    df = feed_dwell_df[feed_dwell_df['count'] <= threshold]
    days = ['Mon', 'Tue','Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    for i in reversed(range(len(days))):
        # Use x instead of y argument for horizontal plot
        fig.add_trace(go.Box(x=df.loc[df['day of week']==days[i], 'count'], name=days[i]))

    # layout - axes labels
    #fig.update_layout(
    #    xaxis_title=metric,
    #    xaxis_rangeslider_visible=True
    #)
    # title
    
    fig.update_layout(
        title="Pedestrian count under threshold by day of week",
        xaxis_title="time",
        yaxis_title="count")
    
    fig.show()

In [52]:
_ = interact(plot_boxplot_count_by_day, 
             threshold=widgets.IntSlider(value=50, min=50, max=1000, step=50, readout_format='d'))

interactive(children=(IntSlider(value=50, description='threshold', max=1000, min=50, step=50), Output()), _dom…

In [53]:
def plot_boxplot_count_by_hour(threshold):
    fig = go.Figure()
    
    #df, byvals, clrs = get_df(groupby)
    df = feed_dwell_df[feed_dwell_df['count'] <= threshold]
    for j in range(7, 21):
        fig.add_trace(go.Box(x=df.loc[df['hour']==j, 'count'], name=j))

    # layout - axes labels
    #fig.update_layout(
    #    xaxis_title=metric,
    #    xaxis_rangeslider_visible=True
    #)
    # title
    
    fig.update_layout(
        title="Pedestrian count under threshold grouped by hour",
        xaxis_title="time",
        yaxis_title="count")
    
    fig.show()

In [54]:
_ = interact(plot_boxplot_count_by_hour, 
             threshold=widgets.IntSlider(value=100, min=50, max=1000, step=50, readout_format='d'))

interactive(children=(IntSlider(value=100, description='threshold', max=1000, min=50, step=50), Output()), _do…